# Backward Differentiation Formula


In [ ]:
using Pkg
Pkg.activate(".")
using SymPy

Now let's make up some variables in a similar way to the Adams Methods

In [ ]:
x1, x2, x3, x4, x5, x6, x7, x8 = symbols("x1, x2, x3, x4, x5, x6, x7, x8")
t1, t2, t3, t4, t5, t6, t7, t8 = symbols("t1, t2, t3, t4, t5, t6, t7, t8")
t = symbols("t")
X = [x1,x2,x3,x4,x5,x6,x7,x8]
T = [t1,t2,t3,t4,t5,t6,t7,t8]
f = symbols("f")
h = symbols("h")

function l(t,j,k)
    l = 1
    for m in 1:k
        if m != j
            l *= ( t - T[m] )/( T[j] - T[m] )
        end        
    end
    return l
end
;

This time we interpolate the $x$ values, instead of the $f(x)$ values.  Then
$$ x(t) \approx p(t) =  \sum_{j=0}^{k} x_j \ell_j(t)$$
So $\dot x \approx \frac{d p}{dt} $, where we simply differentiate $p$ as
$$ \frac{dp}{dt}  = \sum_{j=0}^{k} x_j \frac{d \ell_j(t)}{dt} $$
Putting this back into the differential equation
$$ \dot x = f(x) $$
gives the discrete equation for the next time step
$$ \left.\frac{dp}{dt}\right|_{t_{i+1}} = f(x_{i+1}) $$ 

In [ ]:
n = 3
P = [ X[i]*l(t, i, n) for i in 1:n ] |> sum

In [ ]:
dP = diff(P,t)

Again, let's assume that the time step is constant $h$

In [ ]:
dP1 = dP |> subs(t=>T[n]) |>  subs(t2=>t1+h, t3=>t1+2*h, t4=>t1+3*h, t5=>t1+4*h, t6=>t1+5*h, t7=>t1+6*h, t8=>t1+7*h) |> simplify

Now we can put this discrete approximation of the derivative back into the differential equation (where the $f = f(x_{i+1},t_{i+1})= f(x_3,t_3)$ 

In [ ]:
eq = Eq( dP1, f)

Lastly we can solve this system for $x_3$

In [ ]:
solve(eq, X[n] )[1]

This gives the right-hand-side of the classic BDF2 formula

In [ ]:
Eq( x3, solve(eq, X[n] )[1] )

## variable time step
Now I'm going to work with the variable step size methods for the vBDF.jl example

In [ ]:
n = 3+1
P = [ X[i]*l(t, i, n) for i in 1:n ] |> sum
dP = diff(P,t) |> subs(t=>T[n])
coeffList = [diff(dP, x) |> simplify for x in X[1:n] ]

In [ ]:
@vars h h1 h0
# h= t4 - t3; h1 = t3 - t2; h0 = t2 - t1
hterms = [c |> subs( t4=> h + t3 ) |> subs( t3=> h1 + t2 ) |> subs( t2=> h0 + t1 ) |> expand |> simplify for c in coeffList ]

In [ ]:
sympy.julia_code( hterms[1] ) |> println